In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import zipfile
import io

In [22]:
# Getting the URL for the zip files from the Lyft/ Ford AWS 
response= requests.get('https://s3.amazonaws.com/baywheels-data')
response

<Response [200]>

In [34]:
# writing the index of file_names into a local file
my_folder= r'C:\Users\srini\Udacity\Udacity Ford GoBike'
my_file= 'index2.html'
with open ( os.path.join(my_folder,my_file), 'wb') as file:
    file.write(response.content)

In [35]:
#opening the HTML file using beautiful soup
with open (os.path.join(my_folder,my_file), 'r') as file:
    soup= BeautifulSoup(file)

In [85]:
#finding all the names of the files in the webpage that need to be downloaded
my_list= [str(i).split('>')[1].split('<')[0] for i in soup.find_all('key')]
my_list[:2]

['2017-fordgobike-tripdata.csv.zip', '201801-fordgobike-tripdata.csv.zip']

In [117]:
# Getting the URL for the zip files from the Lyft/ Ford AWS 
my_url= 'https://s3.amazonaws.com/baywheels-data'

for x in my_list[:-1]:
    my_url_mod= my_url+'/'+x   # create a unique URL for each file
    
    response= requests.get(my_url_mod)   
    
    # Downloading and extracting the zipfile
    z= zipfile.ZipFile(io.BytesIO(response.content))  
    z.extractall(os.path.join(my_folder,str(x[:-4])))  # Giving local hardrive path to store the extracted files

In [154]:
# Specifying the data type for each column in the csv file
my_dtype= {'duration_sec':'int', 'start_time': 'str', 'end_time':'str', 'start_station_id':'float',
       'start_station_name':'str', 'start_station_latitude': 'float',
       'start_station_longitude':'float', 'end_station_id':'float', 'end_station_name':'str',
       'end_station_latitude':'float', 'end_station_longitude':'float', 'bike_id':'int', 'user_type':'str',
       'rental_access_method':'object'}

In [157]:
df= pd.read_csv(os.path.join(my_folder, my_list[0][:-4], my_list[0][:-4]))

for i in my_list[:-1]:
    temp= pd.read_csv(os.path.join(my_folder, x[:-4], x[:-4]), dtype= my_dtype) # MAking sure that .zip element is removed
    df= pd.concat([df,temp], sort= False, )
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12193258 entries, 0 to 432353
Data columns (total 14 columns):
duration_sec               int64
start_time                 object
end_time                   object
start_station_id           float64
start_station_name         object
start_station_latitude     float64
start_station_longitude    float64
end_station_id             float64
end_station_name           object
end_station_latitude       float64
end_station_longitude      float64
bike_id                    int64
user_type                  object
rental_access_method       object
dtypes: float64(6), int64(2), object(6)
memory usage: 1.4+ GB


In [159]:
df.shape

(12193258, 14)

There are a total of 12 million rows in the dataset that contains all the ride data made by Ford GoBike from 2017 to 2020.

In [161]:
# Exporting the combined master dataframe to local hard drive
df.to_csv(os.path.join(my_folder,'master.csv'),index=False )

In [162]:
df.head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,rental_access_method
0,80110,2017-12-31 16:57:39.6540,2018-01-01 15:12:50.2450,74.0,Laguna St at Hayes St,37.776435,-122.426244,43.0,San Francisco Public Library (Grove St at Hyde...,37.778768,-122.415929,96,Customer,NaN
1,78800,2017-12-31 15:56:34.8420,2018-01-01 13:49:55.6170,284.0,Yerba Buena Center for the Arts (Howard St at ...,37.784872,-122.400876,96.0,Dolores St at 15th St,37.766210,-122.426614,88,Customer,NaN
2,45768,2017-12-31 22:45:48.4110,2018-01-01 11:28:36.8830,245.0,Downtown Berkeley BART,37.870348,-122.267764,245.0,Downtown Berkeley BART,37.870348,-122.267764,1094,Customer,NaN
3,62172,2017-12-31 17:31:10.6360,2018-01-01 10:47:23.5310,60.0,8th St at Ringold St,37.774520,-122.409449,5.0,Powell St BART Station (Market St at 5th St),37.783899,-122.408445,2831,Customer,NaN
4,43603,2017-12-31 14:23:14.0010,2018-01-01 02:29:57.5710,239.0,Bancroft Way at Telegraph Ave,37.868813,-122.258764,247.0,Fulton St at Bancroft Way,37.867789,-122.265896,3167,Subscriber,NaN


In [167]:
# Finding more missing values in the columns as df.info is not providing the information

df.duration_sec.isna().sum() # no misisng values
df.start_station_id.isna().sum() # there are 7 million missing rows of data for the starting station
df.end_station_id.isna().sum() #there are also 7 million missing rows of data for the ending station
df.start_station_latitude.isna().sum()

0

In [169]:
# Changing the start time and end time to DateTime format
df.start_time= pd.to_datetime(df.start_time)

In [170]:
df.end_time = pd.to_datetime(df.end_time)

In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12193258 entries, 0 to 432353
Data columns (total 14 columns):
duration_sec               int64
start_time                 datetime64[ns]
end_time                   datetime64[ns]
start_station_id           float64
start_station_name         object
start_station_latitude     float64
start_station_longitude    float64
end_station_id             float64
end_station_name           object
end_station_latitude       float64
end_station_longitude      float64
bike_id                    int64
user_type                  object
rental_access_method       object
dtypes: datetime64[ns](2), float64(6), int64(2), object(4)
memory usage: 1.4+ GB


In [172]:
# Adding a duration in min column for better analysis later on
df['duration_min']= df.duration_sec/60